In [1]:
from sklearn.preprocessing import OneHotEncoder
import sys
import tensorflow as tf
import os
import SimpleITK as sitk
import yaml
import numpy as np

In [13]:
with open("444428.yml") as f:
    yamlobj = yaml.load(f)
    model = tf.keras.models.model_from_yaml(yaml.dump(yamlobj))
    
# get patchsize
print("input_shape =", model.input_shape)
print("output_shape =", model.output_shape)
ps = np.array(model.output_shape[1:4])[::-1]
ips = np.array(model.input_shape[1:4])[::-1]
ds = ((ips - ps) / 2).astype(np.int)

print("ps:{}, ips:{}, ds:{}".format(ps, ips, ds))

input_shape = (None, 116, 132, 132, 1)
output_shape = (None, 28, 44, 44, 2)
ps:[44 44 28], ips:[132 132 116], ds:[44 44 44]


In [10]:
def Padding(image, patchsize, imagepatchsize, mirroring = False):
    padfilter = sitk.MirrorPadImageFilter() if mirroring else sitk.ConstantPadImageFilter()
    padfilter.SetPadLowerBound(patchsize.tolist())
    padfilter.SetPadUpperBound(imagepatchsize.tolist())
    padded_image = padfilter.Execute(image)
    return padded_image

def createParentPath(filepath):
    if not os.path.exists(filepath):
        os.makedirs(filepath)
        print("path created")

## creat mask

In [2]:
for i in os.listdir(r"C:\Users\VMLAB\Desktop\kidney\kits19\data"):
    filepath=os.path.join(r"C:\Users\VMLAB\Desktop\kidney\kits19\data",i)#filepath:path of an original image
    filepath=os.path.join(filepath,"segmentation.nii.gz")
    print(filepath)
    savepath=os.path.join(r"C:\Users\VMLAB\Desktop\kidney\kits19\data",i)
    image = sitk.ReadImage(filepath)
    image_array=sitk.GetArrayFromImage(image)
    print("shpe",image_array.shape)
    print("mac",np.max(image_array))
    image_array[image_array>0]=1
    print("shpe",image_array.shape)
    print("mac",np.max(image_array))
    outimage = sitk.GetImageFromArray(image_array)
    outimage.SetOrigin(image.GetOrigin())
    outimage.SetSpacing(image.GetSpacing())
    outimage.SetDirection(image.GetDirection())
    sitk.WriteImage(outimage, savepath+"/mask.mha", True)

C:\Users\VMLAB\Desktop\kidney\kits19\data\case_00000\segmentation.nii.gz
shpe (602, 602, 101)
mac 2
shpe (602, 602, 101)
mac 1
C:\Users\VMLAB\Desktop\kidney\kits19\data\case_00001\segmentation.nii.gz
shpe (523, 523, 100)
mac 2
shpe (523, 523, 100)
mac 1
C:\Users\VMLAB\Desktop\kidney\kits19\data\case_00002\segmentation.nii.gz
shpe (615, 615, 87)
mac 2
shpe (615, 615, 87)
mac 1
C:\Users\VMLAB\Desktop\kidney\kits19\data\case_00003\segmentation.nii.gz
shpe (560, 560, 90)
mac 2
shpe (560, 560, 90)
mac 1
C:\Users\VMLAB\Desktop\kidney\kits19\data\case_00004\segmentation.nii.gz
shpe (639, 639, 85)
mac 2
shpe (639, 639, 85)
mac 1
C:\Users\VMLAB\Desktop\kidney\kits19\data\case_00005\segmentation.nii.gz
shpe (639, 639, 139)
mac 2
shpe (639, 639, 139)
mac 1
C:\Users\VMLAB\Desktop\kidney\kits19\data\case_00006\segmentation.nii.gz
shpe (486, 486, 157)
mac 2
shpe (486, 486, 157)
mac 1
C:\Users\VMLAB\Desktop\kidney\kits19\data\case_00007\segmentation.nii.gz
shpe (615, 615, 61)
mac 2
shpe (615, 615, 61

RuntimeError: Exception thrown in SimpleITK ReadImage: C:\d\VS14-Win64-pkg\SimpleITK\Code\IO\src\sitkImageReaderBase.cxx:99:
sitk::ERROR: The file "C:\Users\VMLAB\Desktop\kidney\kits19\data\kits.json\segmentation.nii.gz" does not exist.

### 8 origan  mask

liver            6   5

spleen           1   1

left_kidney      3   3

right_kidney     2   2

pancreas         11  8

GB(gallbladder)  4   4

aorta            8   6

IVC              9   7

In [29]:
for i in os.listdir(r"D:\comp30\RawData\Training\label"):
    filepath=os.path.join(r"D:\comp30\RawData\Training\label",i)
    filepath=os.path.join(filepath,"label.mha")
    print(filepath)
    savepath=os.path.join(r"D:\comp30\RawData\Training\label",i)
    
    image = sitk.ReadImage(filepath)
    image_array=sitk.GetArrayFromImage(image)
    print("shpe",image_array.shape)
    print("mac",np.max(image_array))
    
    image_array[image_array==1]=1
    image_array[image_array==2]=1
    image_array[image_array==3]=1
    image_array[image_array==4]=1
    image_array[image_array==5]=0
    image_array[image_array==6]=1
    image_array[image_array==7]=0
    image_array[image_array==8]=1
    image_array[image_array==9]=1
    image_array[image_array==10]=0
    image_array[image_array==11]=1
    image_array[image_array==12]=0
    image_array[image_array==13]=0
    
    print("shpe",image_array.shape)
    print("mac",np.max(image_array))
    outimage = sitk.GetImageFromArray(image_array)
    outimage.SetOrigin(image.GetOrigin())
    outimage.SetSpacing(image.GetSpacing())
    outimage.SetDirection(image.GetDirection())
    sitk.WriteImage(outimage, savepath+"/mask_8.mha", True)

D:\comp30\RawData\Training\label\label0001.nii\label.mha
shpe (147, 512, 512)
mac 13
shpe (147, 512, 512)
mac 1
D:\comp30\RawData\Training\label\label0002.nii\label.mha
shpe (139, 512, 512)
mac 13
shpe (139, 512, 512)
mac 1
D:\comp30\RawData\Training\label\label0003.nii\label.mha
shpe (198, 512, 512)
mac 13
shpe (198, 512, 512)
mac 1
D:\comp30\RawData\Training\label\label0004.nii\label.mha
shpe (140, 512, 512)
mac 13
shpe (140, 512, 512)
mac 1
D:\comp30\RawData\Training\label\label0005.nii\label.mha
shpe (117, 512, 512)
mac 13
shpe (117, 512, 512)
mac 1
D:\comp30\RawData\Training\label\label0006.nii\label.mha
shpe (131, 512, 512)
mac 13
shpe (131, 512, 512)
mac 1
D:\comp30\RawData\Training\label\label0007.nii\label.mha
shpe (163, 512, 512)
mac 13
shpe (163, 512, 512)
mac 1
D:\comp30\RawData\Training\label\label0008.nii\label.mha
shpe (148, 512, 512)
mac 13
shpe (148, 512, 512)
mac 1
D:\comp30\RawData\Training\label\label0009.nii\label.mha
shpe (149, 512, 512)
mac 13
shpe (149, 512, 512

### 8 origan label

liver            6   5

spleen           1   1

left_kidney      3   3

right_kidney     2   2

pancreas         11  8

GB(gallbladder)  4   4

aorta            8   6

IVC              9   7


In [30]:
for i in os.listdir(r"D:\comp30\RawData\Training\label"):
    filepath=os.path.join(r"D:\comp30\RawData\Training\label",i)
    filepath=os.path.join(filepath,"label.mha")
    print(filepath)
    savepath=os.path.join(r"D:\comp30\RawData\Training\label",i)
    
    image = sitk.ReadImage(filepath)
    image_array=sitk.GetArrayFromImage(image)
    print("shpe",image_array.shape)
    print("mac",np.max(image_array))
    
    #image_array[image_array==1]=1
    #image_array[image_array==2]=1
    #image_array[image_array==3]=1
    #image_array[image_array==4]=1
    image_array[image_array==5]=0
    image_array[image_array==6]=5
    image_array[image_array==7]=0
    image_array[image_array==8]=6
    image_array[image_array==9]=7
    image_array[image_array==10]=0
    image_array[image_array==11]=8
    image_array[image_array==12]=0
    image_array[image_array==13]=0
    
    print("shpe",image_array.shape)
    print("mac",np.max(image_array))
    outimage = sitk.GetImageFromArray(image_array)
    outimage.SetOrigin(image.GetOrigin())
    outimage.SetSpacing(image.GetSpacing())
    outimage.SetDirection(image.GetDirection())
    sitk.WriteImage(outimage, savepath+"/label_8.mha", True)

In [11]:
for i in os.listdir(r"C:\Users\VMLAB\Desktop\kidney\kits19\data"):
    filepath=os.path.join(r"C:\Users\VMLAB\Desktop\kidney\kits19\data",i)#filepath:path of an original image
    filepath=os.path.join(filepath,"imaging.nii.gz")
    print(filepath)
    savepath=os.path.join(r"C:\Users\VMLAB\Desktop\kidney\kits19\data",i)
    image = sitk.ReadImage(filepath)
    image_array=sitk.GetArrayFromImage(image)
    print("shpe",image_array.shape)
    print("mac",np.max(image_array))
    print("shpe",image_array.shape)
    print("mac",np.max(image_array))
    outimage = sitk.GetImageFromArray(image_array)
    outimage.SetOrigin(image.GetOrigin())
    outimage.SetSpacing(image.GetSpacing())
    outimage.SetDirection(image.GetDirection())
    sitk.WriteImage(outimage, savepath+"/imaging.mha", True)

C:\Users\VMLAB\Desktop\kidney\kits19\data\case_00000\imaging.nii.gz
shpe (602, 602, 101)
mac 1401.9619748553844
shpe (602, 602, 101)
mac 1401.9619748553844
C:\Users\VMLAB\Desktop\kidney\kits19\data\case_00001\imaging.nii.gz
shpe (523, 523, 100)
mac 1359.246689339241
shpe (523, 523, 100)
mac 1359.246689339241


KeyboardInterrupt: 